### Need to generate Adult Data and pass it in self.data
<ul>
    <li> <del>environment virtuel python sur cedar
    <li> <del>Make sure GPU is used
    <li><del> Need ask best params from Rosin</li>
    <li> <del>run Adult
    <li> <del> make sure toSave data from PreProcessing somewhere
    <li> <del>Wrap to take two csv of corresponding original (label)
    <li> <del>Code test testing loop
    <li> <del>Same Encoding for labels and training data
    <li> Remove Sex attribute from training data
    <li> <b>W
</ul>


<li>Is dataloaders right shape
<li>Writing tensor row by row

### Imports

In [1]:
# # Imports
from Modules import utilities as utils
# import datasets as d

# from Stats import Plots as Pl
import tqdm
import time
import csv
import yaml
import json
import torch
import random
import os.path
import warnings
import importlib
import torchvision
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from typing import List
from torch.utils.data import *
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils import data as td
from joblib import Parallel, delayed


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CPU_DEVICE = torch.device("cpu")
GET_VALUE = lambda x: x.to(CPU_DEVICE).data.numpy().reshape(-1)[0]
print(f"\n *** \n Currently running on {device}\n *** \n")
with open('./data/full_original.csv', 'r') as r:
        read = csv.reader(r)
        header = next(read)


 *** 
 Currently running on cpu
 *** 



In [10]:
df = pd.read_csv('./data/full_a=0_E=19.csv')
a = df.describe()
a

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,income
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,41.986344,1.401606e+05,10.262459,0.668482,1.734696,0.174645,38.391589,0.239282
std,13.780262,8.198864e+04,2.419857,0.470764,103.601847,6.891164,10.792135,0.426649
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,31.000000,8.545050e+04,9.000000,0.000000,0.000000,0.000000,34.000000,0.000000
50%,40.000000,1.296565e+05,10.000000,1.000000,0.000000,0.000000,39.000000,0.000000
75%,51.000000,1.791860e+05,13.000000,1.000000,0.000000,0.000000,43.000000,0.000000
max,90.000000,1.202937e+06,16.000000,1.000000,12682.000000,665.000000,99.000000,1.000000


In [11]:
df2 = pd.read_csv('./data/full_original.csv')
b = df2.describe()
b

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,income
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,0.668482,1079.067626,87.502314,40.422382,0.239282
std,13.710510,1.056040e+05,2.570973,0.470764,7452.019058,403.004552,12.391444,0.426649
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.781445e+05,10.000000,1.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.376420e+05,12.000000,1.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,1.000000,99999.000000,4356.000000,99.000000,1.000000


In [13]:
f = a==b

In [14]:
f

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,income
count,True,True,True,True,True,True,True,True
mean,False,False,False,True,False,False,False,True
std,False,False,False,True,False,False,False,True
min,True,True,True,True,True,True,True,True
25%,False,False,True,True,True,True,False,True
50%,False,False,True,True,True,True,False,True
75%,False,False,False,True,True,True,False,True
max,True,False,True,True,False,False,True,True


In [2]:
def tensor_to_df(tensor: torch.Tensor, headers: List[str]) -> pd.DataFrame:
    '''
        Takes in a 2d or 3d tensor and its column list and returns datafram
        if 3d, we assume first dim (dim=0) is the batch size, hence its ignored


    :PARAMS
    tensor: tensor to convert
    headers: list of headers to assign in that order. Must be same size 
                as last dim of tensor parameter.

    '''
    if tensor.shape[-1] != len(headers):
        raise ValueError(f"Tensor's last dimension ({tensor.shape[-1]}) must match headers length ({len(headers)})")

    df = pd.DataFrame (tensor.tolist(), columns=headers)
    return df

In [41]:
a = torch.tensor([[2,3],[5,1],[9,1]])
head = ['abbb','bcccc']
a.tolist()

[[2, 3], [5, 1], [9, 1]]

In [42]:
tensor_to_df(a, head)

,abbb,bcccc
0,2,3
1,5,1
2,9,1


In [3]:
# exp_name = 'manual_test'
# path_to_exp = utils.check_dir_path(f'./experiments/{exp_name}/')
# os.mkdir(path_to_exp)
# Data import & Pre-processing
  
class My_dataLoader:
    def __init__(self, batch_size : int, data_path :str, label_path:str, n_train :int,  label_col_name:str, test_batch_size:int=128):
        '''
            Creates train and test loaders from local files, to be easily used by torch.nn
            
            :batch_size: int for size of training batches
            :data_path: path to csv where data is. 2d file
            :label_path: csv containing labels. line by line equivalent to data_path file
            :n_train: int for the size of training set (assigned randomly)
            :test_batch: size of batches at test time. If none, will be same 
                    as training
            :label_col_name name of columns that contains labels
        '''
        self.batch_size = batch_size
        self.train_size = n_train
        
        df_data = pd.read_csv(data_path)
        df_label = pd.read_csv(label_path)
        
        a = df_data.values
        b = df_label.values
        self.trdata = torch.tensor(a[:self.train_size,:]) # where data is 2d [D_train_size x features]
        self.trlabels = torch.tensor(b[:self.train_size,:]) # also has too be 2d
        self.tedata = torch.tensor(a[self.train_size:,:]) # where data is 2d [D_train_size x features]
        self.telabels = torch.tensor(b[self.train_size:,:]) # also has too be 2d
        
        self.train_dataset = torch.utils.data.TensorDataset(self.trdata, self.trlabels)
        self.test_dataset = torch.utils.data.TensorDataset(self.tedata, self.telabels)

        # Split dataset into train and Test sets
        self.train_loader = torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=1,
            pin_memory=False
        )
        if test_batch_size == 'full':
            test_batch_size = len(self.test_dataset)
            
        self.test_loader = DataLoader(
            self.test_dataset,
            batch_size=test_batch_size,
            num_workers=1,
            pin_memory=False
        )

class PreProcessing:
    def __init__(self, params_file: str):
        '''
            Imports all variables/parameters necessary for preparation of training.
            Looks into params.yaml, then creates dataloader that can be used
            
            params_file: path to file that contains parameters.
                            Needs to follow a specific naming and format
        '''
        
        # Import params
        stream = open(params_file, 'r')
        self.params = yaml.load(stream, yaml.FullLoader)

        import_path = self.params['data_loading']['data_path']['value']
        label_path = self.params['data_loading']['label_path']['value']

        batchSize = self.params['model_params']['batchSize']['value']        
        test_batch_size = self.params['model_params']['test_batch_size']['value']        
        percent_train_set = self.params['model_params']['percent_train_set']['value']
        
        #Make encoding object to transform categorical
        self.data_pp = utils.encode(import_path)
        self.labels_pp = utils.encode(label_path)
        print(f"Saving output dataset under {import_path[:-4]}_NoCat.csv \n")
        self.data_pp.df.to_csv(f"{import_path[:-4]}_NoCat.csv", index=False)
        self.labels_pp.df.to_csv(f"{label_path[:-4]}_NoCat.csv", index=False)

        # MAke sure post processing label and data are of same shape
        if self.data_pp.df.shape != self.data_pp.df.shape:
            print(self.df_data.shape)
            print(self.df_labels.shape)
            raise ValueError("The labels csv and data post-encoding don't have the same shape.")

        self.dataloader = My_dataLoader(batchSize, f"{import_path[:-4]}_NoCat.csv", f"{label_path[:-4]}_NoCat.csv", n_test, "income", test_batch_size)



### Training

In [4]:
# Box

class Autoencoder(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(), nn.Linear(8, 4))#, nn.ReLU(True), nn.Linear(4, 4))
        self.decoder = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(4, 16),
            nn.Sigmoid(),
            nn.Dropout(0.25),
#             nn.Linear(10, 16),
#             nn.ReLU(True),
            nn.Linear(16, 24),
            nn.Sigmoid(), nn.Linear(24, out_dim))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        
        return x
    


def train(model,train_loader, optimizer, loss_fn):
    model.train()
    train_loss = []
    for batch_idx, (inputs, target) in enumerate(train_loader):
      
        #inputs = inputs.view(batchSize, 1,100,100)
        inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs.float())
        loss = loss_fn(output.float(), target.float())
        train_loss.append(loss)
        # Backprop
        loss.backward()
        optimizer.step()

    mean_loss = sum(train_loss) / batch_idx+1
    return mean_loss

def test(model, test_loader, test_loss_fn, last_epoch=False):
    model.eval()
    
    test_loss = 0
    test_size = 0
    with torch.no_grad():
        for inputs, target in test_loader:

            inputs, target = inputs.to(device), target.to(device)
            
            if last_epoch == True:
                data = inputs.tolist()
                with open(f"./experiments/{str.replace(time.ctime()[4:-8], ' ', '_')}-original_testset.csv", 'w', newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(header)
                    writer.writerows(data)
                
            output = model(inputs.float())

            if last_epoch == True:
                data = output.tolist()
                with open(f"./experiments/{str.replace(time.ctime()[4:-8], ' ', '_')}-generated_testset.csv", 'w', newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(header)
                    writer.writerows(data)
            
            test_size += len(inputs.float())
            test_loss += test_loss_fn(output.float(), target.float()).item() 
    test_loss /= test_size

    return test_loss


In [5]:
a = Autoencoder(20,20)

In [6]:
def train_model(experiment_x: PreProcessing, model_type:str='autoencoder'):
    '''
        Takes care of all model training, testing and generation of data
        
        experiment_x: PreProcessing object that contains all data and parameters
    
    '''
    start = time.time()
    wd = experiment_x.params['model_params']['weight_decay']['value']
    learning_rate = experiment_x.params['model_params']['learning_rate']['value']

    num_epochs = int(experiment_x.params['model_params']['num_epochs']['value'])

    in_dim = experiment_x.dataloader.trdata.shape[1]
    out_dim = experiment_x.dataloader.trlabels.shape[1]
    #######

    if model_type == 'autoencoder':
        model = Autoencoder(in_dim, out_dim).to(device)
    train_loss = torch.nn.L1Loss().to(device) # BCEWithLogitsLoss combines sig, so no need activate out layer with softmax
    test_loss_fn =torch.nn.L1Loss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wd)
    test_accuracy = []
    ave_train_loss = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch} running...")

        batch_ave_tr_loss = train(model,experiment_x.dataloader.train_loader, optimizer, train_loss)
        ave_train_loss.append(batch_ave_tr_loss)

        last = False
        if epoch+1 == num_epochs: # then save the test batch input and output for metrics
            last = True
        loss = test(model, experiment_x.dataloader.test_loader, test_loss_fn, last)

        print(f"Epoch {epoch} complete. Test Loss: {loss:.4f} \n")      
        test_accuracy.append(loss)


    a = experiment_x.params['data_loading']['data_path']['value'][36:].split('/')[0]

    fm = open(f"./experiments/{str.replace(time.ctime(), ' ', '_')}-{a}.pth", "wb")
    torch.save(model.state_dict(), fm)

    with open(f"./experiments/{str.replace(time.ctime(), ' ', '_')}-{a}.txt", 'w+') as f:
        f.write(f"Epochs: {num_epochs} \n")
        f.write(f"Learning Rate: {learning_rate} \n")
        f.write(f"weight decay: {wd}\n")
        f.write(f"Training Loss: {str(train_loss)}\n")
        f.write(f"Test Loss: {str(test_loss_fn)} \n")
        f.write(f"Optimizer: {str(optimizer)}\n")
        f.write(f"Train loss values: {str(ave_train_loss)} \n")
        f.write(f"Test loss values: {str(test_accuracy)}\n")

    end = time.time()
    print(f"Training on {num_epochs} epochs completed in {(end-start)/60} minutes.")
    
experiment = PreProcessing("params.yaml")

# train_model(experiment)

KeyError: 'batchSize'

In [8]:
def from_dummies(data, categories, prefix_sep='-'):
    """
        Convert encoded columns into original ones
    """
#     data = self.df
#     categories = self.cat_clm
    cat_was_num = {'sex': dtype('int64'), 'income': dtype('int64')}
    out = data.copy()
    for l in categories:
        cols = data.filter(regex="^{}{}".format(l, prefix_sep), axis=1).columns
        labs = [cols[i].split(prefix_sep)[-1] for i in range(cols.shape[0])]
        import pdb;pdb.set_trace()
        out[l] = pd.Categorical(np.array(labs)[np.argmax(data[cols].values, axis=1)])
        out.drop(cols, axis=1, inplace=True)
        if l in cat_was_num.keys():
            out[l] = out[l].astype(cat_was_num[l])

    return out

a = from_dummies(df, unique_cat_cols)

NameError: name 'dtype' is not defined

In [20]:
import pickle
from Modules import datasets as d

prm_file = "/home/jc/Desktop/udem_H20/thesis_research/gansan/Experiments/AdultTotalC/Exp_1/Prm/"
path_to_exp = "/home/jc/Desktop/udem_H20/thesis_research/reconstructor/experiments/manual_test/"

d_i = d.Preprocessing('/home/jc/Desktop/udem_H20/thesis_research/reconstructor/data/full_original.csv')
d_i.load_parameters(prmPath=prm_file)
d_i.transform()
d_i.df

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,sex=0,sex=1,income=0,income=1,...,native-country=Portugal,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia
0,0.342466,0.157823,0.800000,0.000000,0.0,0.112245,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.493151,0.093023,0.800000,0.076881,0.0,0.397959,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.205479,0.048121,0.800000,0.000000,0.0,0.346939,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.575342,0.090175,0.666667,0.000000,0.0,0.397959,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.041096,0.145665,0.533333,0.000000,0.0,0.397959,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.602740,0.130317,0.400000,0.000000,0.0,0.357143,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48838,0.082192,0.249141,0.600000,0.000000,0.0,0.397959,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48839,0.328767,0.118744,0.600000,0.000000,0.0,0.397959,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48840,0.328767,0.115632,0.533333,0.000000,0.0,0.500000,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
plt.plot(np.arange(1,num_epochs+1), np.array(test_accuracy))
plt.xlabel("Epochs")
plt.ylabel("L1 Loss")
plt.title("Test Loss")
plt.savefig(f"./experiments/{str.replace(time.ctime(), ' ', '_')}-{a}_test-loss.png")
plt.show()

In [7]:

plt.plot(np.arange(1,num_epochs+1), np.array(train_losses))
plt.xlabel("Epochs")
plt.ylabel("L1 Loss")
plt.title("Train Loss - average per epoch")
plt.savefig(f"./experiments/{str.replace(time.ctime(), ' ', '_')}-{a}_train-loss.png")
plt.show()

NameError: name 'train_losses' is not defined